# Increasing the Maximum Message Size for gRPC


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Helm](https://github.com/kubernetes/helm)
 - [Minikube](https://github.com/kubernetes/minikube) version v0.24.0 or greater
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core```)

# Create Cluster

Start minikube and ensure custom resource validation is activated and there is 5G of memory. 

An example start command using the kvm2 driver would look like:
```
minikube start --vm-driver kvm2 --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC
```

# Setup

In [1]:
!kubectl create namespace seldon

namespace/seldon created


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


In [3]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


# Install Helm

In [4]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/janis/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


## Start seldon-core

Install the custom resource definition

In [7]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Tue Dec 18 12:17:56 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     1s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  1s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          1s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        1s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s


NOTES:
NOTES: TODO




In [8]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon \
    --set ambassador.enabled=true

NAME:   seldon-core
LAST DEPLOYED: Tue Dec 18 12:17:59 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/Pod(related)
NAME                                                 READY  STATUS             RESTARTS  AGE
seldon-core-ambassador-56cb8fc595-zj2lj              0/1    ContainerCreating  0         0s
seldon-core-seldon-apiserver-59978fbf45-8wwzn        0/1    ContainerCreating  0         0s
seldon-core-seldon-cluster-manager-5449b8dd5f-8xrt6  0/1    ContainerCreating  0         0s
seldon-core-redis-567db9ddcf-779cl                   0/1    ContainerCreating  0         0s

==> v1/ServiceAccount
NAME    SECRETS  AGE
seldon  1        1s

==> v1beta1/Role
NAME          AGE
seldon-local  1s
ambassador    1s

==> v1/RoleBinding
NAME    AGE
seldon  1s

==> v1beta1/RoleBinding
NAME        AGE
ambassador  1s

==> v1/Service
NAME                          TYPE       CLUSTER-IP     EXTERNAL-IP  PORT(S)                        AGE
seldon-core-ambassador-admin  NodePort   10.98.239.177  <no

Check all services are running before proceeding.

In [9]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador 

Waiting for deployment "seldon-core-seldon-cluster-manager" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out
Waiting for deployment "seldon-core-ambassador" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward to API Gateway**

REST:

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

GRPC:

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8004:5000
```

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8005:8080
```

In [10]:
!pygmentize resources/model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-model"
    },
    "spec": {
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                        ],
                     

## Create Seldon Deployment

Deploy the runtime graph to kubernetes.

In [11]:
!kubectl apply -f resources/model.json -n seldon

seldondeployment.machinelearning.seldon.io/seldon-model created


Get the status of the SeldonDeployment. **When ready the replicasAvailable should be 1**.

In [13]:
!kubectl get seldondeployments seldon-model -o jsonpath='{.status}' -n seldon

map[state:Available predictorStatus:[map[name:test-deployment-example-7cd068f replicas:1 replicasAvailable:1]]]

## Get predictions - no grpx max message size

In [14]:
from seldon_utils import *
API_GATEWAY_REST="localhost:8003"
API_GATEWAY_GRPC="localhost:8004"
API_AMBASSADOR="localhost:8005"

Send a small request which should suceed.

In [15]:
grpc_request_api_gateway('oauth-key','oauth-secret',None,API_GATEWAY_REST,API_GATEWAY_GRPC,10)

{"access_token":"2ac70653-8b36-46f6-b5e8-611c3c090029","token_type":"bearer","expires_in":43199,"scope":"read write"}


meta {
  puid: "uqb9mt5qi3790bvvbgmus5oklr"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.0892753869230776
  }
}

Test via Ambassador

In [16]:
grpc_request_ambassador("seldon-model",None,API_AMBASSADOR,data_size=10,rows=1)

meta {
  puid: "6nlf542sb7mgvukpsot2l361bh"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08619452296855779
  }
}

Send a large request which will be above the default gRPC message size and will fail.

In [17]:
grpc_request_api_gateway('oauth-key','oauth-secret',None,API_GATEWAY_REST,API_GATEWAY_GRPC,1000000)

{"access_token":"2ac70653-8b36-46f6-b5e8-611c3c090029","token_type":"bearer","expires_in":43187,"scope":"read write"}


_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.CANCELLED
	details = "Received RST_STREAM with error code 8"
	debug_error_string = "{"created":"@1545135751.395194140","description":"Error received from peer","file":"src/core/lib/surface/call.cc","file_line":1095,"grpc_message":"Received RST_STREAM with error code 8","grpc_status":1}"
>

In [18]:
grpc_request_ambassador("seldon-model",None,API_AMBASSADOR,data_size=1000000,rows=1)

_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "upstream connect error or disconnect/reset before headers"
	debug_error_string = "{"created":"@1545135755.480635396","description":"Error received from peer","file":"src/core/lib/surface/call.cc","file_line":1095,"grpc_message":"upstream connect error or disconnect/reset before headers","grpc_status":14}"
>

In [19]:
!kubectl delete -f resources/model.json

seldondeployment.machinelearning.seldon.io "seldon-model" deleted


In [20]:
!helm delete seldon-core --purge

release "seldon-core" deleted


# Allowing larger gRPC messages

Recreate seldon-core with extra annotation for the API Gateway

In [21]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon \
    --set ambassador.enabled=true \
    -f grpc_size_values.yaml

NAME:   seldon-core
LAST DEPLOYED: Tue Dec 18 12:22:05 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME                                DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-core-ambassador              1        1        1           0          0s
seldon-core-seldon-apiserver        1        1        1           0          0s
seldon-core-seldon-cluster-manager  1        1        1           0          0s
seldon-core-redis                   1        1        1           0          0s

==> v1/Pod(related)
NAME                                                 READY  STATUS             RESTARTS  AGE
seldon-core-ambassador-56cb8fc595-495hc              0/1    ContainerCreating  0         0s
seldon-core-ambassador-56cb8fc595-zj2lj              1/1    Terminating        0         4m5s
seldon-core-seldon-apiserver-59978fbf45-8wwzn        0/1    Terminating        0         4m5s
seldon-core-seldon-apiserver-64d4fb745f-sqgsp        0/1    ContainerCreati

Wait for seldon core deployment to be running

In [22]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador 

deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out
Waiting for deployment "seldon-core-ambassador" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


Now we change our SeldonDeployment to include a annotation for max grpx message size.

In [23]:
!pygmentize resources/model_grpc_size.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-model"
    },
    "spec": {
        "annotations": {
	    "seldon.io/grpc-max-message-size":"10000000"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier_grpc:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                     

In [24]:
!kubectl create -f resources/model_grpc_size.json -n seldon

seldondeployment.machinelearning.seldon.io/seldon-model created


### **Make sure you rerun the port forwards above as a new seldon core API gateway and Ambassador will have started**

In [26]:
!kubectl get seldondeployments seldon-model -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[replicasAvailable:0 name:test-deployment-grpc-size-fd60a01 replicas:1]] state:Creating]

Send a large message. This time it should succeed.

In [31]:
grpc_request_api_gateway('oauth-key','oauth-secret',None,API_GATEWAY_REST,API_GATEWAY_GRPC,data_size=1000000,rows=1)

{"access_token":"500e9e17-a1f1-4174-a14c-cf988f673eb4","token_type":"bearer","expires_in":43125,"scope":"read write"}


meta {
  puid: "3k8cfubi9vcoogsup3fk1itdna"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier_grpc:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08192989817777528
  }
}

Send a request via ambassador. This should also succeed.

In [32]:
grpc_request_ambassador("seldon-model",None,API_AMBASSADOR,data_size=1000000,rows=1)

meta {
  puid: "9sir96lidq9gees0bb70s1uqdb"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier_grpc:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08191949643488104
  }
}